In [2]:
import pandas as pd
import folium
from haversine import haversine
import numpy as np
import seaborn as sns
import json
import os
from copy import deepcopy 
import geopandas as gpd
import pathlib
import matplotlib.pyplot as plt
input_path = pathlib.Path('./data')

In [8]:
어린이보호구역 = pd.read_csv(input_path.joinpath('9.오산시_어린이보호구역.csv'),encoding="utf-8")
법정경계_읍면동 = gpd.read_file(input_path.joinpath('33.오산시_법정경계(읍면동).geojson'),encoding="utf-8")
어린이교통사고_격자 = gpd.read_file(input_path.joinpath('2.오산시_어린이교통사고_격자.geojson'),encoding="utf-8")

In [9]:
어린이보호구역

시설종류         시설명 CCTV설치여부  CCTV설치대수  보호구역도로폭     보호구역_경도    보호구역_위도
0   특수학교        성심학교        N         0       25  127.017521  37.176762
1   초등학교      가수초등학교        Y         1       12  127.047957  37.148103
2   초등학교      광성초등학교        Y         2       12  127.039709  37.184197
3   초등학교      대호초등학교        Y         3       12  127.057889  37.159106
4   초등학교      매홀초등학교        Y         2        6  127.063282  37.162749
..   ...         ...      ...       ...      ...         ...        ...
86  어린이집    시립세마어린이집        Y         1       12  127.044916  37.184608
87  어린이집     미래숲어린이집        Y         1        6  127.044384  37.145326
88  어린이집   시립은여울어린이집        N         0       20  127.066860  37.170952
89  어린이집     설리반어린이집        N         0        5  127.061041  37.164127
90  어린이집  세교복지타운어린이집        Y         1       15  127.044835  37.177752

[91 rows x 7 columns]

In [10]:
어린이교통사고_격자['cut']=0
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=2]=1
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=5]=2
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=7]=3
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=10]=4

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the 

In [11]:
def check_overlaps(row):
    global 어린이보호구역
    
    if not row.cut:
        return False

    for row2 in 어린이보호구역.iloc:
        [lng,lat] = row2[-2:]
        
        if haversine((lat,lng),row['geometry'].centroid.coords[0][::-1],unit="m") <= 300:
            return True
    return False
            
어린이교통사고_격자['overlapped'] = 어린이교통사고_격자.apply(check_overlaps,axis=1)

In [12]:
center =[37.15222, 127.07056]
zoom =13

m = folium.Map(location=center, zoom_start=zoom)

folium.GeoJson(
    법정경계_읍면동,
    name = "동 경계",
    style_function=lambda feature: {
        'color': '#000000',
        'weight': 2,
        'opacity':0.2,
    }
).add_to(m)

for line in 어린이보호구역.iloc:
    [lng,lat] = line[-2:]
    
    folium.Circle(
    location = [lat,lng],
    radius=300,
    stroke=False,
    fill=True,
    fill_color='green',
    fill_opacity=0.6).add_to(m)
    
folium.GeoJson(
    어린이교통사고_격자[어린이교통사고_격자.cut > 0],
    name = '교통사고 2회 이상',
    style_function=lambda feature: {
        'color': 'red',
        'weight': 3,
        'opacity':0.7,
    }
).add_to(m)


후보_격자 = 어린이교통사고_격자[(어린이교통사고_격자.cut > 0)&(어린이교통사고_격자.overlapped == False)]

folium.GeoJson(
    후보_격자,
    name = '중심좌표 기준 후보 격자',
    style_function=lambda feature: {
        'color': 'yellow',
        'weight': 3,
        'opacity':0.7,
    }
).add_to(m)

folium.LayerControl().add_to(m)

m

In [13]:
후보_격자.head(5)

gid  accident_cnt  \
1665  다사593100             2   
1738  다사594093             2   
1751  다사594106             2   
1752  다사594107             3   
1882  다사596077             2   

                                               geometry  cut  overlapped  
1665  MULTIPOLYGON (((127.04144 37.18790, 127.04144 ...    1       False  
1738  MULTIPOLYGON (((127.04261 37.18159, 127.04260 ...    1       False  
1751  MULTIPOLYGON (((127.04254 37.19331, 127.04253 ...    1       False  
1752  MULTIPOLYGON (((127.04253 37.19421, 127.04253 ...    1       False  
1882  MULTIPOLYGON (((127.04495 37.16718, 127.04494 ...    1       False

In [14]:
후보_격자

gid  accident_cnt  \
1665  다사593100             2   
1738  다사594093             2   
1751  다사594106             2   
1752  다사594107             3   
1882  다사596077             2   
1982  다사597102             2   
2004  다사598049             2   
2266  다사601085             2   
2335  다사602079             2   
2637  다사606089             2   
2710  다사607089             3   
2902  다사610057             5   
2910  다사610065             2   
3098  다사612101             6   
3139  다사613065             2   
3289  다사615054             2   
3340  다사616033             2   
3358  다사616051             3   
3495  다사618051             2   
3681  다사621048            10   
3737  다사622056             4   
3738  다사622057             2   
3781  다사623056             2   
3782  다사623057             3   
3823  다사624056             2   
3849  다사625042             5   
3850  다사625043             5   
3863  다사625056             6   
3932  다사627042             4   
4076  다사630067             2   
4155  다사632067             3   

                                               geometry  cut  overlapped  
1665  MULTIPOLYGON (((127.04144 37.18790, 127.04144 ...    1       False  
1738  MULTIPOLYGON (((127.04261 37.18159, 127.04260 ...    1       False  
1751  MULTIPOLYGON (((127.04254 37.19331, 127.04253 ...    1       False  
1752  MULTIPOLYGON (((127.04253 37.19421, 127.04253 ...    1       False  
1882  MULTIPOLYGON (((127.04495 37.16718, 127.04494 ...    1       False  
1982  MULTIPOLYGON (((127.04594 37.18972, 127.04593 ...    1       False  
2004  MULTIPOLYGON (((127.04735 37.14195, 127.04735 ...    1       False  
2266  MULTIPOLYGON (((127.05054 37.17441, 127.05053 ...    1       False  
2335  MULTIPOLYGON (((127.05170 37.16901, 127.05169 ...    1       False  
2637  MULTIPOLYGON (((127.05615 37.17804, 127.05614 ...    1       False  
2710  MULTIPOLYGON (((127.05728 37.17804, 127.05727 ...    1       False  
2902  MULTIPOLYGON (((127.06082 37.14921, 127.06082 ...    2       False  
2910  MULTIPOLYGON (((127.06078 37.15642, 127.06077 ...    1       False  
3098  MULTIPOLYGON (((127.06285 37.18888, 127.06284 ...    2       False  
3139  MULTIPOLYGON (((127.06416 37.15643, 127.06415 ...    1       False  
3289  MULTIPOLYGON (((127.06647 37.14652, 127.06646 ...    1       False  
3340  MULTIPOLYGON (((127.06770 37.12760, 127.06770 ...    1       False  
3358  MULTIPOLYGON (((127.06761 37.14382, 127.06760 ...    1       False  
3495  MULTIPOLYGON (((127.06986 37.14383, 127.06986 ...    1       False  
3681  MULTIPOLYGON (((127.07325 37.14114, 127.07325 ...    4       False  
3737  MULTIPOLYGON (((127.07434 37.14836, 127.07433 ...    1       False  
3738  MULTIPOLYGON (((127.07433 37.14926, 127.07433 ...    1       False  
3781  MULTIPOLYGON (((127.07547 37.14836, 127.07546 ...    1       False  
3782  MULTIPOLYGON (((127.07546 37.14926, 127.07546 ...    1       False  
3823  MULTIPOLYGON (((127.07659 37.14836, 127.07659 ...    1       False  
3849  MULTIPOLYGON (((127.07779 37.13575, 127.07778 ...    2       False  
3850  MULTIPOLYGON (((127.07778 37.13665, 127.07778 ...    2       False  
3863  MULTIPOLYGON (((127.07772 37.14837, 127.07771 ...    2       False  
3932  MULTIPOLYGON (((127.08004 37.13576, 127.08003 ...    1       False  
4076  MULTIPOLYGON (((127.08329 37.15830, 127.08329 ...    1       False  
4155  MULTIPOLYGON (((127.08555 37.15831, 127.08554 ...    1       False

In [16]:
후보_격자.shape

(31, 5)

- 후보 격자 생성 완료

### 앞에서 MSE가 제일 작았던 Adaboost Model FItting

In [26]:
total = pd.read_csv('total_sub.csv',index_col='Unnamed: 0')
total.sample(6) #(345,61)

accident_cnt  고령인구_count_100  생산가능인구_count_100  신호등_sum   유동인구_sum  \
138             1              28               167        0  17.304833   
19              2              51               388        6  27.411833   
280             1             155               619        2  17.901417   
93              1              35               324        6  13.736667   
170             4              93               497        8  30.168333   
114             1              11               269        0  68.937129   

     유소년인구_count_100  유소년인구_count_200  주정차_count_300  차량등록_count_200  \
138               13               69              9               0   
19                91              463              5               0   
280              109              411              7               0   
93                63              214              5               0   
170              129              383             10               0   
114                7               13              8               0   

     학원수_count_100  ...  학구.raw오산원일초통학구역  학구.raw오산초가수초공동통학구역  학구.raw오산초통학구역  \
138              0  ...                0                   0              0   
19               0  ...                0                   0              0   
280              0  ...                0                   0              0   
93               0  ...                0                   0              0   
170              0  ...                0                   0              0   
114              0  ...                0                   0              0   

     학구.raw운산초통학구역  학구.raw운암초통학구역  학구.raw원동초통학구역  학구.raw필봉초통학구역  \
138              0              0              0              0   
19               0              0              0              0   
280              1              0              0              0   
93               0              0              0              0   
170              0              0              0              0   
114              0              0              0              0   

     학구.raw화성초통학구역  정류장_count_200  총인구_count_100  
138              0              2            209  
19               0              1            530  
280              0              2            883  
93               0              2            425  
170              0              5            719  
114              0              6            295  

[6 rows x 81 columns]

In [27]:
total.columns

Index(['accident_cnt', '고령인구_count_100', '생산가능인구_count_100', '신호등_sum',
       '유동인구_sum', '유소년인구_count_100', '유소년인구_count_200', '주정차_count_300',
       '차량등록_count_200', '학원수_count_100', '학원수_count_300', '횡단보도_sum',
       '방지턱_count', '방지턱_count_100', '방지턱_count_200', '불법주정차_cam_count',
       '어린이집_count', '어린이집_count_100', '어린이집_count_200', '유치원_count',
       '유치원_count_100', '유치원_count_200', '정류장_count_100', '정류장_count_300',
       '초등학교_count', '초등학교_count_100', '초등학교_count_200', '동.raw가수동',
       '동.raw가장동', '동.raw갈곶동', '동.raw고현동', '동.raw궐동', '동.raw금암동', '동.raw내삼미동',
       '동.raw누읍동', '동.raw벌음동', '동.raw부산동', '동.raw서동', '동.raw서랑동', '동.raw세교동',
       '동.raw수청동', '동.raw양산동', '동.raw오산동', '동.raw외삼미동', '동.raw원동', '동.raw은계동',
       '동.raw지곶동', '동.raw청학동', '동.raw청호동', '동.raw탑동', '학구.raw가수초통학구역',
       '학구.raw광성초양산초병점초공동통학구역', '학구.raw광성초통학구역', '학구.raw금암초통학구역',
       '학구.raw대호초성산초공동통학구역', '학구.raw대호초통학구역', '학구.raw매홀초삼미분교수청초필봉초공동통학구역',
       '학구.raw매홀초삼미분교장통학구역', '학구.raw매홀초통학구역', '학

In [28]:
'''
X = pd.DataFrame(total.drop(['gid','geometry','가장동','갈곶동','고현동','궐동','금암동','내삼미동','부산동','누읍동','벌음동','부산동','서동','서랑동',
                                '세교동','수청동','양산동','오산동','외삼미동','원동','은계동','지곶동','청학동','청호동','탑동',
                                '광성초양산초병점초공동통학구역','광성초통학구역','금암초통학구역','대호초성산초공동통학구역',
                                '대호초통학구역','매홀초삼미분교수청초필봉초공동통학구역','매홀초삼미분교장통학구역','매홀초통학구역',
                                '문시초통학구역','성산초통학구역','성호초운암초공동통학구역','성호초통학구역','세미초통학구역','수청초통학구역',
                                '양산초통학구역', '오산고현초오산원당초공동통학구역','오산고현초통학구역','오산대원초원동초공동통학구역','오산대원초통학구역',
                                '오산원당초통학구역','오산원일초통학구역','오산초가수초공동통학구역','오산초통학구역','운산초통학구역','운암초통학구역',
                                '원동초통학구역','필봉초통학구역','화성초통학구역'],axis=1))
'''
X = pd.DataFrame(total.drop(['accident_cnt'],axis=1))
cols = X.columns
Y = pd.Series(total["accident_cnt"])
X.shape , Y.shape

((345, 80), (345,))

### - split validation set

In [29]:
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
m_scaler = MinMaxScaler()
X = scaler.fit_transform(X)

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=2021)
x_train.shape , x_test.shape

((276, 80), (69, 80))

In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import cross_val_score

In [31]:
my_scorer = make_scorer(mean_squared_error,greater_is_better=False)

kf = KFold(n_splits=10,shuffle=True,random_state=2021)

In [32]:
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor(n_estimators=200,learning_rate=0.1,loss="exponential",random_state=2021)
ada.fit(x_train, y_train)

Rscore = ada.score(x_test,y_test)
score = cross_val_score(ada,x_test,y_test,cv=kf,scoring=my_scorer)  
print("MSE :",score.mean()*(-1))

MSE : 0.47954329369814125


In [33]:
total = pd.read_csv('total_sub.csv',index_col='Unnamed: 0')
total.head()

accident_cnt  고령인구_count_100  생산가능인구_count_100  신호등_sum   유동인구_sum  \
1             1               0                 0        0  43.435676   
2             1               0                 0        0   9.633417   
3             1               0                 0        0   0.215500   
4             1               0                13        0   0.662000   
5             1               0                 0        8   8.104333   

   유소년인구_count_100  유소년인구_count_200  주정차_count_300  차량등록_count_200  \
1                0                0              2               0   
2                0                0              2               0   
3                0                0              2               0   
4                0                0              0               1   
5                0                0              6               0   

   학원수_count_100  ...  학구.raw오산원일초통학구역  학구.raw오산초가수초공동통학구역  학구.raw오산초통학구역  \
1              0  ...                0                   0              0   
2              0  ...                0                   0              0   
3              0  ...                0                   0              0   
4              0  ...                0                   0              0   
5              0  ...                0                   0              0   

   학구.raw운산초통학구역  학구.raw운암초통학구역  학구.raw원동초통학구역  학구.raw필봉초통학구역  학구.raw화성초통학구역  \
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   
5              0              0              0              0              0   

   정류장_count_200  총인구_count_100  
1              2              0  
2              0              0  
3              1              0  
4              0             13  
5              4              0  

[5 rows x 81 columns]

In [34]:
X = pd.DataFrame(total.drop(['accident_cnt'],axis=1))
cols = X.columns
Y = pd.Series(total["accident_cnt"])
X.shape , Y.shape

((345, 80), (345,))

In [35]:
pred = ada.predict(X)
pred

array([1.57142857, 1.55813953, 1.56923077, 1.55813953, 2.2278481 ,
       1.67142857, 1.83653846, 1.68888889, 2.27717391, 1.67142857,
       1.67142857, 2.21568627, 1.74736842, 3.45454545, 3.66666667,
       1.75      , 3.43023256, 1.81300813, 3.43023256, 1.89473684,
       1.90909091, 1.57731959, 1.56097561, 2.57777778, 1.82      ,
       3.        , 1.56923077, 1.90909091, 3.09090909, 2.86875   ,
       3.07865169, 2.58974359, 1.57446809, 2.2278481 , 2.2278481 ,
       3.45454545, 3.00862069, 1.72727273, 1.67142857, 3.00862069,
       1.82432432, 1.57446809, 3.5       , 3.5       , 1.99453552,
       2.24561404, 1.90909091, 1.74074074, 3.5       , 1.91304348,
       2.23076923, 3.25974026, 2.86875   , 1.90909091, 1.55813953,
       1.87623762, 3.43023256, 3.40350877, 1.90909091, 1.87623762,
       1.93835616, 2.21568627, 1.97777778, 1.55813953, 1.79310345,
       2.90361446, 1.90909091, 1.93835616, 1.87623762, 1.87623762,
       3.33333333, 1.82432432, 1.55813953, 1.79310345, 3.40350

In [36]:
temp = pd.read_csv('total_dat.csv') # R 파일 맨앞에 있던 파일!

In [37]:
total['gid'] = temp.gid

In [38]:
total['pred_score'] = pred

In [39]:
total

accident_cnt  고령인구_count_100  생산가능인구_count_100  신호등_sum   유동인구_sum  \
1               1               0                 0        0  43.435676   
2               1               0                 0        0   9.633417   
3               1               0                 0        0   0.215500   
4               1               0                13        0   0.662000   
5               1               0                 0        8   8.104333   
..            ...             ...               ...      ...        ...   
341             1               0                 0        0  18.696500   
342             1             120               872        0  74.573667   
343             1              31               256        3  63.209333   
344             1               0                 0        0  85.970583   
345             1               9                13        0  47.054500   

     유소년인구_count_100  유소년인구_count_200  주정차_count_300  차량등록_count_200  \
1                  0                0              2               0   
2                  0                0              2               0   
3                  0                0              2               0   
4                  0                0              0               1   
5                  0                0              6               0   
..               ...              ...            ...             ...   
341                0              240              4               0   
342              176              543              7               0   
343                6              231              4               0   
344                0                0              2               0   
345                0               42              4               0   

     학원수_count_100  ...  학구.raw오산초통학구역  학구.raw운산초통학구역  학구.raw운암초통학구역  \
1                0  ...              0              0              0   
2                0  ...              0              0              0   
3                0  ...              0              0              0   
4                0  ...              0              0              0   
5                0  ...              0              0              0   
..             ...  ...            ...            ...            ...   
341              0  ...              0              0              0   
342              0  ...              0              0              0   
343              0  ...              0              0              0   
344              0  ...              0              0              0   
345              0  ...              0              0              0   

     학구.raw원동초통학구역  학구.raw필봉초통학구역  학구.raw화성초통학구역  정류장_count_200  \
1                0              0              0              2   
2                0              0              0              0   
3                0              0              0              1   
4                0              0              0              0   
5                0              0              0              4   
..             ...            ...            ...            ...   
341              0              0              0              1   
342              0              0              0              5   
343              0              0              0              4   
344              0              0              0              2   
345              0              0              0              0   

     총인구_count_100       gid  pred_score  
1                0  다사564096    1.571429  
2                0  다사566102    1.558140  
3                0  다사583093    1.569231  
4               13  다사584072    1.558140  
5                0  다사585098    2.227848  
..             ...       ...         ...  
341              0  다사633049    1.721154  
342           1168  다사634036    1.928571  
343            296  다사635067    2.589744  
344              0  다사636036    1.571429  
345             22       NaN    1.652174  

[345 rows x 83 columns]

In [41]:
candidate = 후보_격자
candidate.head()

gid  accident_cnt  \
1665  다사593100             2   
1738  다사594093             2   
1751  다사594106             2   
1752  다사594107             3   
1882  다사596077             2   

                                               geometry  cut  overlapped  
1665  MULTIPOLYGON (((127.04144 37.18790, 127.04144 ...    1       False  
1738  MULTIPOLYGON (((127.04261 37.18159, 127.04260 ...    1       False  
1751  MULTIPOLYGON (((127.04254 37.19331, 127.04253 ...    1       False  
1752  MULTIPOLYGON (((127.04253 37.19421, 127.04253 ...    1       False  
1882  MULTIPOLYGON (((127.04495 37.16718, 127.04494 ...    1       False

In [42]:
total = pd.DataFrame(total.drop(['accident_cnt'],axis=1)) # 우리가 궁금한건 후보격자의 accident_cnt와의 비교이므로 total에서 accident_cnt 삭제
total.head()

고령인구_count_100  생산가능인구_count_100  신호등_sum   유동인구_sum  유소년인구_count_100  \
1               0                 0        0  43.435676                0   
2               0                 0        0   9.633417                0   
3               0                 0        0   0.215500                0   
4               0                13        0   0.662000                0   
5               0                 0        8   8.104333                0   

   유소년인구_count_200  주정차_count_300  차량등록_count_200  학원수_count_100  \
1                0              2               0              0   
2                0              2               0              0   
3                0              2               0              0   
4                0              0               1              0   
5                0              6               0              0   

   학원수_count_300  ...  학구.raw오산초통학구역  학구.raw운산초통학구역  학구.raw운암초통학구역  \
1              0  ...              0              0              0   
2              0  ...              0              0              0   
3              0  ...              0              0              0   
4              0  ...              0              0              0   
5              0  ...              0              0              0   

   학구.raw원동초통학구역  학구.raw필봉초통학구역  학구.raw화성초통학구역  정류장_count_200  총인구_count_100  \
1              0              0              0              2              0   
2              0              0              0              0              0   
3              0              0              0              1              0   
4              0              0              0              0             13   
5              0              0              0              4              0   

        gid  pred_score  
1  다사564096    1.571429  
2  다사566102    1.558140  
3  다사583093    1.569231  
4  다사584072    1.558140  
5  다사585098    2.227848  

[5 rows x 82 columns]

In [43]:
dat = pd.merge(candidate,total,left_on='gid',right_on='gid',how='left')
dat.head()

gid  accident_cnt                                           geometry  \
0  다사593100             2  MULTIPOLYGON (((127.04144 37.18790, 127.04144 ...   
1  다사594093             2  MULTIPOLYGON (((127.04261 37.18159, 127.04260 ...   
2  다사594106             2  MULTIPOLYGON (((127.04254 37.19331, 127.04253 ...   
3  다사594107             3  MULTIPOLYGON (((127.04253 37.19421, 127.04253 ...   
4  다사596077             2  MULTIPOLYGON (((127.04495 37.16718, 127.04494 ...   

   cut  overlapped  고령인구_count_100  생산가능인구_count_100  신호등_sum    유동인구_sum  \
0    1       False               0                 0        0  117.510833   
1    1       False             176              1242        0   36.420667   
2    1       False               0                 0        0   50.763850   
3    1       False               0                 0        3   57.970750   
4    1       False              78               602        0    0.724455   

   유소년인구_count_100  ...  학구.raw오산초가수초공동통학구역  학구.raw오산초통학구역  학구.raw운산초통학구역  \
0                0  ...                   0              0              0   
1              264  ...                   1              0              0   
2                0  ...                   0              0              0   
3                0  ...                   0              0              0   
4              144  ...                   1              0              0   

   학구.raw운암초통학구역  학구.raw원동초통학구역  학구.raw필봉초통학구역  학구.raw화성초통학구역  정류장_count_200  \
0              0              0              0              0              4   
1              0              0              0              0              4   
2              0              0              0              0              2   
3              0              0              0              0              2   
4              0              0              0              0              2   

   총인구_count_100  pred_score  
0              0    1.820000  
1           1682    1.909091  
2              0    1.574468  
3              0    2.227848  
4            824    1.824324  

[5 rows x 86 columns]

In [44]:
dat['total_score']=dat.pred_score-dat.accident_cnt

In [211]:
dat.to_csv('ada_totalscore.csv')

In [45]:
top20 = dat.sort_values(by='total_score').head(20)

In [214]:
top20.to_csv('ada_top_20.csv')

In [46]:
ada_top20 = top20[['accident_cnt','total_score']]
ada_top20

accident_cnt  total_score
19            10    -5.000000
13             6    -4.396825
27             6    -4.186992
11             5    -3.441860
20             4    -2.076271
30             3    -1.441860
26             5    -1.333333
25             5    -1.333333
10             3    -1.252632
3              3    -0.772152
28             4    -0.569767
2              2    -0.425532
15             2    -0.259259
12             2    -0.259259
24             2    -0.206897
14             2    -0.201835
0              2    -0.180000
4              2    -0.175676
16             2    -0.132653
1              2    -0.090909